# *0. preparation of environment and datasets*

## 0.1 python environment

### 0.1.1 install python dependency

In [1]:
# !pip3 install torch
# !pip3 install numpy
# !pip3 install xgboost
# !pip3 install seaborn
# !pip3 install catboost
# !pip3 install torchvision

### 0.1.2 import the library

In [1]:
import os
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import mpl_toolkits.mplot3d
from scipy import stats
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

import xgboost as xgb
from catboost import CatBoostRegressor

## 0.2 change to the work path

In [2]:
# root_file_path = "/home/extension/kaggle/IEEE-CIS_Fraud_Detection"
root_file_path = "/home/hdd/kaggle/ieee_cis"

In [3]:
os.chdir(root_file_path)

## 0.3 read the files

### 0.3.1 training set

In [11]:
train_transcation_set = pd.read_csv("train_transaction.csv.zip")

In [12]:
train_identity_set = pd.read_csv("train_identity.csv.zip")

### 0.3.2 test set

In [4]:
test_transaction_set = pd.read_csv("test_transaction.csv.zip")

In [5]:
test_identity_set = pd.read_csv("test_identity.csv.zip")

# *1. data preprocessing*

## 1.0. data previewing

In [14]:
train_transcation_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Columns: 394 entries, TransactionID to V339
dtypes: float64(376), int64(4), object(14)
memory usage: 1.7+ GB


In [15]:
train_identity_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144233 entries, 0 to 144232
Data columns (total 41 columns):
TransactionID    144233 non-null int64
id_01            144233 non-null float64
id_02            140872 non-null float64
id_03            66324 non-null float64
id_04            66324 non-null float64
id_05            136865 non-null float64
id_06            136865 non-null float64
id_07            5155 non-null float64
id_08            5155 non-null float64
id_09            74926 non-null float64
id_10            74926 non-null float64
id_11            140978 non-null float64
id_12            144233 non-null object
id_13            127320 non-null float64
id_14            80044 non-null float64
id_15            140985 non-null object
id_16            129340 non-null object
id_17            139369 non-null float64
id_18            45113 non-null float64
id_19            139318 non-null float64
id_20            139261 non-null float64
id_21            5159 non-null float64
id_2

## 1.1 merge sets

### 1.1.1 training set

In [13]:
df_train = pd.merge(train_transcation_set, train_identity_set, how="left", on=["TransactionID"])

In [18]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 590540 entries, 0 to 590539
Columns: 434 entries, TransactionID to DeviceInfo
dtypes: float64(399), int64(4), object(31)
memory usage: 1.9+ GB


In [ ]:
df_train.describ()

### 1.1.2 testing set

In [6]:
df_test = pd.merge(test_transaction_set, test_identity_set, how="left", on=["TransactionID"])

In [21]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 506691 entries, 0 to 506690
Columns: 433 entries, TransactionID to DeviceInfo
dtypes: float64(399), int64(3), object(31)
memory usage: 1.6+ GB


In [ ]:
df_test.describ()

## 1.2 data analyzation and visualization

### 1.2.0 visualization

#### 1.2.0.0 preparation

In [22]:
le = LabelEncoder()

#### 1.2.0.1 preparation

In [23]:
def visualization(df_train, total_data_entry):
    for tmp_column in df_train.columns:
        print("Column Name is {}:".format(tmp_column))
        if df_train[tmp_column].dtype != np.object:
    #         df_train[tmp_column].plot(kind='line', label=tmp_column)
            df_train.plot(kind="scatter", x="TransactionID", y=tmp_column)
            plt.legend(loc='upper left')
        else:
            tmp_label = df_train[tmp_column].unique()
            tmp_ = pd.isnull(tmp_label)
            tmp_label = tmp_label[np.where(tmp_ == False)]        
            tmp_count_ = []

            for tmp in tmp_label:
                tmp_count_.append(df_train.loc[df_train[tmp_column] == tmp][tmp_column].count())

            print(tmp_label)
            print(tmp_count_)
            plt.bar(tmp_label,tmp_count_,width=0.3)
            plt.xticks(range(len(tmp_label)),tmp_label,rotation=90)

        plt.show()
        
        tmp_count = df_train[tmp_column].count()
        tmp_proportion = 1 - tmp_count / total_data_entry

        print("Missing data proportion is {}".format(1 - tmp_count / total_data_entry))
        print("-------------------------------------")

#### 1.2.0.2 probability

In [24]:
def probability_plot(input_list, log_flag=False):
    fig = plt.figure()
  
    if log_flag:
        input_list = np.log10(input_list)
    else:
        pass

    res = stats.probplot(input_list, plot=plt)
    plt.show()

#### 1.2.0.3 distribution

In [25]:
def distribution(input_array):
    sns.distplot(input_array, fit=norm)
    (mu, sigma) = norm.fit(input_array)
    print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#### 1.2.0.4 relational heatmap

In [26]:
def encode(df_train):
    for tmp_column in df_train.columns:
        if df_train[tmp_column].dtype != np.object:
            pass
        else:
            print(tmp_column)
            df_train[tmp_column] = le.fit_transform(df_train[tmp_column])

In [27]:
def relational_heatmap(df_train):
    encode(df_train)
    sns.heatmap(df_train.corr(), vmax=0.9, square=True)

### 1.2.1 nan value

#### 1.2.1.0 preparation

In [28]:
print(type(df_train["DeviceType"].dtype))

<class 'numpy.dtype'>


#### 1.2.1.1 proportion of nan value by columns

In [ ]:
def nan_value(column_nan_threshold, row_nan_threshold, df):
    num_entry, num_field = df.shape
    
    null_df = df.isnull()
    
    nan_proportion_column = np.sum(null_df == True) / num_entry
    useful_column = np.where(nan_proportion_column < column_nan_threshold)
    useful_column = df.columns[useful_column[0]]
    
    nan_proportion_row = np.sum(null_df[useful_column] == True, axis=1) / num_field
    row_index = np.where(nan_proportion_row < row_nan_threshold)
    
    return useful_column, row_index

In [30]:
delete_nan_threshold = 0.4

In [31]:
nan_pproportion_column = np.sum(null_df == True) / total_data_entry

In [32]:
usage_column = np.where(nan_pproportion_column < delete_nan_threshold)

In [33]:
usage_column = df_train.columns[usage_column[0]]

#### 1.2.1.2 proportion of nan value by row

In [34]:
delete_nan_threshold_row = 0.2

In [35]:
nan_pproportion_row = np.sum(null_df[usage_column] == True, axis=1) / total_column

In [36]:
index_row = np.where(nan_pproportion_row < delete_nan_threshold_row)

In [37]:
nan_train = df_train.loc[index_row[0]][usage_column]

#### 1.2.1.3 fill nan value by std

In [38]:
nan_fill_dict = {}

In [39]:
for tmp_column in nan_train.columns:
    if nan_train[tmp_column].dtype != np.object:
        nan_fill_dict[tmp_column] = nan_train[tmp_column].std()
        nan_train[tmp_column].fillna(value=nan_train[tmp_column].std(), inplace=True)
    else:
        tmp_label = nan_train[tmp_column].unique()
        tmp_ = pd.isnull(tmp_label)
        tmp_label = tmp_label[np.where(tmp_ == False)]        
        tmp_count_ = []

        for tmp in tmp_label:
            tmp_count_.append(nan_train.loc[df_train[tmp_column] == tmp][tmp_column].count())
            
        index = np.argmax(tmp_count_)
        nan_train[tmp_column].fillna(value=tmp_label[index], inplace=True)
        nan_fill_dict[tmp_column] = tmp_label[index]

In [ ]:
visualization(nan_train, nan_train.shape[0])

### 2.2 encode string data

### 2.3 normalization


In [ ]:
def normalization(data_frame, feature_colnmns):
    np_list_train = data_frame[feature_columns].to_numpy()

      for index, tmp_row in enumerate(np_list_train):
            np_list_train[index] = tmp_row / np.linalg.norm(tmp_row, ord=2)
   
      return np_list_train

### 2.4 labelling

In [41]:
label_collection = nan_train.columns

In [43]:
train_label = label_collection.drop("isFraud")

In [44]:
train_y_label = "isFraud"

## *3. model training*

### 3.1 dataset split

In [46]:
def create_validation(partition, trainset, testset):
    return train_test_split(trainset, testset, test_size=partition)

# kfold = StratifiedKFold(n_splits=10, shuffle=True)

### 3.2 training

#### 3.2.1 model selection

##### 3.2.1.1 catboost

In [ ]:
learning_rate = [0.1, 0.01, 0.001]
depth = [7, 8, 9]

param_dict = dict(learning_rate=learning_rate, depth=depth)

model = CatBoostRegressor()

##### 3.2.1.2 xgboost


In [ ]:
learning_rate = [0.1, 0.01, 0.001]
n_estimators = [150, 300, 450]
max_depth = [7, 8, 9]

param_dict = dict(learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth)

gbm = xgb.XGBRegressor(max_depth=3, n_estimators=300, learning_rate=0.05).fit(np_list_train, train_y)

#### 3.2.2 loss function selection

In [ ]:
def self_loss(calcul_result, predict_result):
    grad = abs(predict_result - calcul_result)
    hess = np.power(np.abs(grad), 0.5)
    return grad, hess

#### 3.2.3 gridsearch

In [ ]:
grid_search = GridSearchCV(model, param_dict)
grid_search.fit(np_list_train, train_y)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_score_

#### 3.2.4 training

In [47]:
train_x = nan_train[train_label]
train_y = nan_train["isFraud"]
eval_set = None

In [48]:
train_x, valid_x, train_y, valid_y = create_validation(0.2, train_x, train_y)
eval_set = (valid_x, valid_y)

In [49]:
train_model = CatBoostRegressor(learning_rate=0.001, logging_level='Verbose',iterations=10000, depth=7)
train_model.fit(train_x, train_y, eval_set=eval_set,plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.1823202	test: 0.1827202	best: 0.1827202 (0)	total: 252ms	remaining: 42m
1:	learn: 0.1822788	test: 0.1826789	best: 0.1826789 (1)	total: 345ms	remaining: 28m 44s
2:	learn: 0.1822373	test: 0.1826378	best: 0.1826378 (2)	total: 433ms	remaining: 24m 4s
3:	learn: 0.1821958	test: 0.1825965	best: 0.1825965 (3)	total: 534ms	remaining: 22m 14s
4:	learn: 0.1821536	test: 0.1825543	best: 0.1825543 (4)	total: 628ms	remaining: 20m 55s
5:	learn: 0.1821123	test: 0.1825130	best: 0.1825130 (5)	total: 722ms	remaining: 20m 2s
6:	learn: 0.1820707	test: 0.1824716	best: 0.1824716 (6)	total: 842ms	remaining: 20m 1s
7:	learn: 0.1820294	test: 0.1824305	best: 0.1824305 (7)	total: 931ms	remaining: 19m 22s
8:	learn: 0.1819887	test: 0.1823899	best: 0.1823899 (8)	total: 1.01s	remaining: 18m 46s
9:	learn: 0.1819473	test: 0.1823492	best: 0.1823492 (9)	total: 1.11s	remaining: 18m 30s
10:	learn: 0.1819055	test: 0.1823076	best: 0.1823076 (10)	total: 1.2s	remaining: 18m 12s
11:	learn: 0.1818647	test: 0.1822665	b

92:	learn: 0.1787345	test: 0.1791543	best: 0.1791543 (92)	total: 9.21s	remaining: 16m 21s
93:	learn: 0.1786976	test: 0.1791176	best: 0.1791176 (93)	total: 9.31s	remaining: 16m 21s
94:	learn: 0.1786618	test: 0.1790817	best: 0.1790817 (94)	total: 9.39s	remaining: 16m 19s
95:	learn: 0.1786255	test: 0.1790459	best: 0.1790459 (95)	total: 9.49s	remaining: 16m 18s
96:	learn: 0.1785895	test: 0.1790103	best: 0.1790103 (96)	total: 9.59s	remaining: 16m 18s
97:	learn: 0.1785534	test: 0.1789743	best: 0.1789743 (97)	total: 9.68s	remaining: 16m 18s
98:	learn: 0.1785177	test: 0.1789387	best: 0.1789387 (98)	total: 9.78s	remaining: 16m 18s
99:	learn: 0.1784823	test: 0.1789034	best: 0.1789034 (99)	total: 9.87s	remaining: 16m 16s
100:	learn: 0.1784472	test: 0.1788686	best: 0.1788686 (100)	total: 9.95s	remaining: 16m 15s
101:	learn: 0.1784113	test: 0.1788330	best: 0.1788330 (101)	total: 10s	remaining: 16m 14s
102:	learn: 0.1783758	test: 0.1787976	best: 0.1787976 (102)	total: 10.1s	remaining: 16m 14s
103:	l

182:	learn: 0.1757092	test: 0.1761442	best: 0.1761442 (182)	total: 17.8s	remaining: 15m 53s
183:	learn: 0.1756774	test: 0.1761128	best: 0.1761128 (183)	total: 17.9s	remaining: 15m 52s
184:	learn: 0.1756462	test: 0.1760816	best: 0.1760816 (184)	total: 17.9s	remaining: 15m 52s
185:	learn: 0.1756159	test: 0.1760513	best: 0.1760513 (185)	total: 18s	remaining: 15m 51s
186:	learn: 0.1755848	test: 0.1760205	best: 0.1760205 (186)	total: 18.1s	remaining: 15m 51s
187:	learn: 0.1755536	test: 0.1759894	best: 0.1759894 (187)	total: 18.2s	remaining: 15m 51s
188:	learn: 0.1755223	test: 0.1759582	best: 0.1759582 (188)	total: 18.3s	remaining: 15m 51s
189:	learn: 0.1754923	test: 0.1759283	best: 0.1759283 (189)	total: 18.4s	remaining: 15m 51s
190:	learn: 0.1754612	test: 0.1758972	best: 0.1758972 (190)	total: 18.5s	remaining: 15m 50s
191:	learn: 0.1754293	test: 0.1758655	best: 0.1758655 (191)	total: 18.6s	remaining: 15m 51s
192:	learn: 0.1753983	test: 0.1758347	best: 0.1758347 (192)	total: 18.7s	remaining

272:	learn: 0.1730772	test: 0.1735274	best: 0.1735274 (272)	total: 27.2s	remaining: 16m 9s
273:	learn: 0.1730495	test: 0.1735001	best: 0.1735001 (273)	total: 27.3s	remaining: 16m 8s
274:	learn: 0.1730232	test: 0.1734739	best: 0.1734739 (274)	total: 27.4s	remaining: 16m 8s
275:	learn: 0.1729959	test: 0.1734470	best: 0.1734470 (275)	total: 27.5s	remaining: 16m 8s
276:	learn: 0.1729689	test: 0.1734199	best: 0.1734199 (276)	total: 27.6s	remaining: 16m 7s
277:	learn: 0.1729408	test: 0.1733922	best: 0.1733922 (277)	total: 27.7s	remaining: 16m 7s
278:	learn: 0.1729146	test: 0.1733658	best: 0.1733658 (278)	total: 27.8s	remaining: 16m 7s
279:	learn: 0.1728873	test: 0.1733386	best: 0.1733386 (279)	total: 27.9s	remaining: 16m 7s
280:	learn: 0.1728604	test: 0.1733122	best: 0.1733122 (280)	total: 28s	remaining: 16m 7s
281:	learn: 0.1728332	test: 0.1732851	best: 0.1732851 (281)	total: 28.1s	remaining: 16m 6s
282:	learn: 0.1728060	test: 0.1732582	best: 0.1732582 (282)	total: 28.1s	remaining: 16m 6s
2

...

9629:	learn: 0.1448090	test: 0.1457183	best: 0.1457183 (9629)	total: 17m 55s	remaining: 41.3s
9630:	learn: 0.1448083	test: 0.1457177	best: 0.1457177 (9630)	total: 17m 55s	remaining: 41.2s
9631:	learn: 0.1448080	test: 0.1457174	best: 0.1457174 (9631)	total: 17m 55s	remaining: 41.1s
9632:	learn: 0.1448074	test: 0.1457172	best: 0.1457172 (9632)	total: 17m 55s	remaining: 41s
9633:	learn: 0.1448066	test: 0.1457162	best: 0.1457162 (9633)	total: 17m 55s	remaining: 40.9s
9634:	learn: 0.1448059	test: 0.1457154	best: 0.1457154 (9634)	total: 17m 55s	remaining: 40.8s
9635:	learn: 0.1448048	test: 0.1457143	best: 0.1457143 (9635)	total: 17m 56s	remaining: 40.6s
9636:	learn: 0.1448042	test: 0.1457136	best: 0.1457136 (9636)	total: 17m 56s	remaining: 40.5s
9637:	learn: 0.1448038	test: 0.1457133	best: 0.1457133 (9637)	total: 17m 56s	remaining: 40.4s
9638:	learn: 0.1448034	test: 0.1457129	best: 0.1457129 (9638)	total: 17m 56s	remaining: 40.3s
9639:	learn: 0.1448027	test: 0.1457123	best: 0.1457123 (9639)	

9719:	learn: 0.1447520	test: 0.1456623	best: 0.1456623 (9719)	total: 18m 7s	remaining: 31.3s
9720:	learn: 0.1447518	test: 0.1456621	best: 0.1456621 (9720)	total: 18m 7s	remaining: 31.2s
9721:	learn: 0.1447512	test: 0.1456617	best: 0.1456617 (9721)	total: 18m 7s	remaining: 31.1s
9722:	learn: 0.1447510	test: 0.1456615	best: 0.1456615 (9722)	total: 18m 7s	remaining: 31s
9723:	learn: 0.1447506	test: 0.1456611	best: 0.1456611 (9723)	total: 18m 7s	remaining: 30.9s
9724:	learn: 0.1447502	test: 0.1456607	best: 0.1456607 (9724)	total: 18m 7s	remaining: 30.8s
9725:	learn: 0.1447498	test: 0.1456602	best: 0.1456602 (9725)	total: 18m 7s	remaining: 30.6s
9726:	learn: 0.1447494	test: 0.1456599	best: 0.1456599 (9726)	total: 18m 7s	remaining: 30.5s
9727:	learn: 0.1447492	test: 0.1456597	best: 0.1456597 (9727)	total: 18m 8s	remaining: 30.4s
9728:	learn: 0.1447482	test: 0.1456586	best: 0.1456586 (9728)	total: 18m 8s	remaining: 30.3s
9729:	learn: 0.1447480	test: 0.1456585	best: 0.1456585 (9729)	total: 18m

9808:	learn: 0.1447047	test: 0.1456182	best: 0.1456182 (9808)	total: 18m 18s	remaining: 21.4s
9809:	learn: 0.1447038	test: 0.1456176	best: 0.1456176 (9809)	total: 18m 18s	remaining: 21.3s
9810:	learn: 0.1447029	test: 0.1456166	best: 0.1456166 (9810)	total: 18m 18s	remaining: 21.2s
9811:	learn: 0.1447023	test: 0.1456158	best: 0.1456158 (9811)	total: 18m 18s	remaining: 21s
9812:	learn: 0.1447018	test: 0.1456152	best: 0.1456152 (9812)	total: 18m 18s	remaining: 20.9s
9813:	learn: 0.1447013	test: 0.1456149	best: 0.1456149 (9813)	total: 18m 18s	remaining: 20.8s
9814:	learn: 0.1447006	test: 0.1456139	best: 0.1456139 (9814)	total: 18m 18s	remaining: 20.7s
9815:	learn: 0.1446999	test: 0.1456131	best: 0.1456131 (9815)	total: 18m 19s	remaining: 20.6s
9816:	learn: 0.1446995	test: 0.1456127	best: 0.1456127 (9816)	total: 18m 19s	remaining: 20.5s
9817:	learn: 0.1446991	test: 0.1456124	best: 0.1456124 (9817)	total: 18m 19s	remaining: 20.4s
9818:	learn: 0.1446988	test: 0.1456121	best: 0.1456121 (9818)	

9897:	learn: 0.1446510	test: 0.1455693	best: 0.1455693 (9897)	total: 18m 28s	remaining: 11.4s
9898:	learn: 0.1446506	test: 0.1455690	best: 0.1455690 (9898)	total: 18m 29s	remaining: 11.3s
9899:	learn: 0.1446500	test: 0.1455686	best: 0.1455686 (9899)	total: 18m 29s	remaining: 11.2s
9900:	learn: 0.1446491	test: 0.1455678	best: 0.1455678 (9900)	total: 18m 29s	remaining: 11.1s
9901:	learn: 0.1446484	test: 0.1455671	best: 0.1455671 (9901)	total: 18m 29s	remaining: 11s
9902:	learn: 0.1446474	test: 0.1455664	best: 0.1455664 (9902)	total: 18m 29s	remaining: 10.9s
9903:	learn: 0.1446470	test: 0.1455661	best: 0.1455661 (9903)	total: 18m 29s	remaining: 10.8s
9904:	learn: 0.1446464	test: 0.1455655	best: 0.1455655 (9904)	total: 18m 29s	remaining: 10.6s
9905:	learn: 0.1446459	test: 0.1455650	best: 0.1455650 (9905)	total: 18m 30s	remaining: 10.5s
9906:	learn: 0.1446454	test: 0.1455646	best: 0.1455646 (9906)	total: 18m 30s	remaining: 10.4s
9907:	learn: 0.1446445	test: 0.1455635	best: 0.1455635 (9907)	

9986:	learn: 0.1445978	test: 0.1455205	best: 0.1455205 (9986)	total: 18m 39s	remaining: 1.46s
9987:	learn: 0.1445972	test: 0.1455200	best: 0.1455200 (9987)	total: 18m 39s	remaining: 1.34s
9988:	learn: 0.1445971	test: 0.1455200	best: 0.1455200 (9988)	total: 18m 40s	remaining: 1.23s
9989:	learn: 0.1445964	test: 0.1455194	best: 0.1455194 (9989)	total: 18m 40s	remaining: 1.12s
9990:	learn: 0.1445958	test: 0.1455188	best: 0.1455188 (9990)	total: 18m 40s	remaining: 1.01s
9991:	learn: 0.1445954	test: 0.1455185	best: 0.1455185 (9991)	total: 18m 40s	remaining: 897ms
9992:	learn: 0.1445948	test: 0.1455180	best: 0.1455180 (9992)	total: 18m 40s	remaining: 785ms
9993:	learn: 0.1445943	test: 0.1455174	best: 0.1455174 (9993)	total: 18m 40s	remaining: 673ms
9994:	learn: 0.1445937	test: 0.1455170	best: 0.1455170 (9994)	total: 18m 40s	remaining: 561ms
9995:	learn: 0.1445929	test: 0.1455163	best: 0.1455163 (9995)	total: 18m 41s	remaining: 449ms
9996:	learn: 0.1445925	test: 0.1455160	best: 0.1455160 (9996

# *4. Evaluation*

In [51]:
nan_test = df_test[usage_column.drop("isFraud")]

In [53]:
for tmp_field in nan_fill_dict:
    if tmp_field == "isFraud":
        continue
    
    nan_test[tmp_field].fillna(nan_fill_dict[tmp_field], inplace=True)

In [54]:
encode(nan_test)

ProductCD
card4
card6


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


P_emaildomain
M6


In [55]:
prediction = train_model.predict(nan_test)

In [56]:
submission = pd.DataFrame({ 'TransactionID': nan_test["TransactionID"],
                            'isFraud': prediction})

In [57]:
submission.to_csv("submission.csv", index=False)

In [58]:
submission.shape

(506691, 2)